In [15]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import os

In [46]:
csv_surfaces_path = 'tmp/REEFMOD.6.8_GBR/geo_DHW.csv'

In [47]:
geo_df = pd.read_csv(csv_surfaces_path)

In [48]:
geo_df

,Reef_ID,Nick_ID,LAT,LON,UNIQUE_ID,LABEL_ID,GBR_NAME,OldReefName,AREA_DESCR,Shelf_position,AIMS_sector,Reference_Area_km2,Geom_2D_Area_km2,GeomCH_2D_Area_km2,Geom_3D_Area_km2,GeomCH_3D_Area_km2,DHW
0,1,42,-10.7795,142.5916,10330100104,10-330,U/N Reef,U/N Reef (10-330),Far Northern,1,1,2.149739,0.223079,0.214122,0.258400,0.249400,[ 0.49999999 1.98999996 4.3799999 0.919999...
1,2,42,-10.7966,142.7106,10331100104,10-331,Harrington Shoal,Harrington Shoal (10-331),Far Northern,1,1,1.099287,0.254949,0.009859,0.336819,0.000802,[ 0.64999999 2.02999995 3.94999991 1.139999...
2,3,42,-10.8243,142.5736,10332100104,10-332,U/N Reef,U/N Reef (10-332),Far Northern,1,1,0.972122,0.001757,0.001149,0.002000,0.001572,[ 0.49999999 1.96999996 4.3399999 0.929999...
3,4,42,-10.8354,142.5766,10333100104,10-333,U/N Reef,U/N Reef (10-333),Far Northern,1,1,0.452624,0.000320,0.000209,0.000300,0.000236,[ 0.49999999 1.96999996 4.3399999 0.929999...
4,5,42,-10.8199,142.6871,10334100104,10-334,Four Fathom Patches,Four Fathom Patches (10-334),Far Northern,1,1,1.675575,0.015122,0.009865,0.015514,0.012174,[ 0.79999998 1.99999996 4.05999991 1.129999...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3801,3802,1266,-23.2053,151.8877,23045102104,23-045b,North Reef (South),North Reef (South) (23-045b),Mackay/Capricorn,3,11,0.308734,0.308734,0.192424,1.577302,1.104642,[0.62999999 1.17999997 5.53999988 0. 0...
3802,3803,1271,-23.2461,151.7826,23046100104,23-046,Tryon Reef,Tryon Reef (23-046),Mackay/Capricorn,3,11,3.533203,3.328520,2.859234,10.021177,8.264400,[0.61999999 1.72999996 6.06999986 0. 0...
3803,3804,1271,-23.2572,151.7618,23047100104,23-047,Brew Shoal,Brew Shoal (23-047),Mackay/Capricorn,3,11,4.270397,1.933965,0.849395,3.993430,1.906000,[0.63999999 1.73999996 5.83999987 0. 0...
3804,3805,1274,-23.2695,151.9403,23048100104,23-048,Broomfield Reef,Broomfield Reef (23-048),Mackay/Capricorn,3,11,15.165481,11.544010,8.583335,23.533580,17.695450,[0.63999999 1.13999997 5.04999989 0. 0...


In [49]:
geo_df = geo_df.set_index(geo_df['Reef_ID'])

In [51]:
ref_surfaces = geo_df.GeomCH_3D_Area_km2
ref_square_sides = np.sqrt(ref_surfaces)
ref_dist_to_walk = ref_square_sides / np.sqrt(2)
bearings_deg = np.array([45, 135, 225, 315, 45])

In [56]:
def get_new_coordinates(lat1, lon1, distance_in_meters, bearing_degrees):
    # Earth's radius in meters
    R = 6371000
    rad2deg = 180. / np.pi
    deg2rad = 1. / rad2deg

    # Convert latitude and longitude to radians
    lat1_rad = lat1 * deg2rad
    lon1_rad = lon1 * deg2rad
    bearing_rad = bearing_degrees * deg2rad

    # Convert distance to radians
    dist_rad = distance_in_meters / R

    # Compute latitude and longitude of destination point
    lat2_rad = np.arcsin(np.sin(lat1_rad) * np.cos(dist_rad) +
                         np.cos(lat1_rad) * np.sin(dist_rad) * np.cos(bearing_rad))

    lon2_rad = lon1_rad + np.arctan2(np.sin(bearing_rad) * np.sin(dist_rad) * np.cos(lat1_rad),
                                     np.cos(dist_rad) - np.sin(lat1_rad) * np.sin(lat2_rad))

    # Convert latitude and longitude back to decimal degrees
    lat2 = lat2_rad * rad2deg
    lon2 = lon2_rad * rad2deg

    return lat2, lon2

In [39]:
OUTPUT_NAME = 'dhw'
CSV_IN_PATH = 'tmp/REEFMOD.6.8_GBR/geo_DHW.csv'
JSON_OUT_PATH = 'geojson_dhw/dhw'
RADII_PATH = 'reef_radii.csv'

In [41]:
outputs_df = pd.read_csv(CSV_IN_PATH, header=0)

In [42]:
outputs_df.head()

,Reef_ID,Nick_ID,LAT,LON,UNIQUE_ID,LABEL_ID,GBR_NAME,OldReefName,AREA_DESCR,Shelf_position,AIMS_sector,Reference_Area_km2,Geom_2D_Area_km2,GeomCH_2D_Area_km2,Geom_3D_Area_km2,GeomCH_3D_Area_km2,DHW
0,1,42,-10.7795,142.5916,10330100104,10-330,U/N Reef,U/N Reef (10-330),Far Northern,1,1,2.149739,0.223079,0.214122,0.258400,0.249400,[ 0.49999999 1.98999996 4.3799999 0.919999...
1,2,42,-10.7966,142.7106,10331100104,10-331,Harrington Shoal,Harrington Shoal (10-331),Far Northern,1,1,1.099287,0.254949,0.009859,0.336819,0.000802,[ 0.64999999 2.02999995 3.94999991 1.139999...
2,3,42,-10.8243,142.5736,10332100104,10-332,U/N Reef,U/N Reef (10-332),Far Northern,1,1,0.972122,0.001757,0.001149,0.002000,0.001572,[ 0.49999999 1.96999996 4.3399999 0.929999...
3,4,42,-10.8354,142.5766,10333100104,10-333,U/N Reef,U/N Reef (10-333),Far Northern,1,1,0.452624,0.000320,0.000209,0.000300,0.000236,[ 0.49999999 1.96999996 4.3399999 0.929999...
4,5,42,-10.8199,142.6871,10334100104,10-334,Four Fathom Patches,Four Fathom Patches (10-334),Far Northern,1,1,1.675575,0.015122,0.009865,0.015514,0.012174,[ 0.79999998 1.99999996 4.05999991 1.129999...


In [37]:
outputs_df = outputs_df.astype(float)

ValueError: could not convert string to float: '10-330'

In [38]:
outputs_df.dtypes

Reef_ID                 int64
Nick_ID                 int64
LAT                   float64
LON                   float64
UNIQUE_ID               int64
LABEL_ID               object
GBR_NAME               object
OldReefName            object
AREA_DESCR             object
Shelf_position          int64
AIMS_sector             int64
Reference_Area_km2    float64
Geom_2D_Area_km2      float64
GeomCH_2D_Area_km2    float64
Geom_3D_Area_km2      float64
GeomCH_3D_Area_km2    float64
dtype: object

In [43]:
output_meta = {
    'n_timesteps': len(outputs_df.columns[:-2]),
    'n_reefs': len(outputs_df['Reef_ID'])
}

KeyError: 0

In [44]:
lon, lat = outputs_df.LON, outputs_df.LAT

In [29]:
# build JSON :
# root level : a list of length n_timesteps
#   level 1 : a list of objects of the form
#   {'x': longitude, 'y': latitude, 'label': OUTPUT_NAME, 'value': a float value}
json_root = {
    'type': 'FeatureCollection',
    'crs': {
        'type': 'name',
        'properties': {
            'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'
        }
    },
    'meta': {
        'name': OUTPUT_NAME,
    }
}

nyears = len(outputs_df.DHW[0])

for t in range(nyears):
    t_str = t + 1985
    path = JSON_OUT_PATH + f't_{t_str}.geojson'
    feature_list = []
    for reef in outputs_df.Reef_ID:
        feature_list.append({
            'type': 'Feature',
            'properties': {
                'reef_id': reef,
                'label': OUTPUT_NAME,
                'radius': 1000. * np.random.rand(),
                'value': outputs_df.DHW[t],
            },
            'geometry': {
                'type': 'Point',
                'coordinates': [lon[reef], lat[reef]]
            }
        })

    json_root['features'] = feature_list
    json_root['meta']['time_step'] = t
    json_root['meta']['IPCC_scenario'] = 'IPCC_scenario_name'
    with open(path, 'w') as out:
        json.dump(json_root, out, indent=2)

In [ ]:
json_root = {
    'type': 'FeatureCollection',
    'crs': {
        'type': 'name',
        'properties': {
            'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'
        }
    },
    'meta': {
        'name': OUTPUT_NAME,
    }
}

for t in range(output_meta['n_timesteps']):
    path = JSON_OUT_PATH + f't_{t}.geojson'
    feature_list = []
    for reef in range(output_meta['n_reefs']):
        feature_list.append({
            'type': 'Feature',
            'properties': {
                'reef_id': reef,
                'label': OUTPUT_NAME,
                'radius': 1000. * np.random.rand(),
                'value': outputs_df[t][reef],
            },
            'geometry': {
                'type': 'Point',
                'coordinates': [lat[reef], lon[reef]]
            }
        })

    json_root['features'] = feature_list
    json_root['meta']['time_step'] = t
    json_root['meta']['IPCC_scenario'] = 'IPCC_scenario_name'
    with open(path, 'w') as out:
        json.dump(json_root, out, indent=2)